##### Импорты

In [22]:
import hashlib
import math

from IPython.display import HTML, display

### MD5

#### Шаг основного цикла вычисления хеша
![Alt text](image.png)

#### Одна итерация цикла раунда
![Alt text](image-1.png)

#### Раундовые функции
| № | Обозначение | Формула |
| - | :-: | - |
| 1 | F | F(B, C, D) = (B ∧ C) ∨ (¬B ∧ D) |
| 2 | G | G(B, C, D) = (B ∧ D) ∨ (¬D ∧ C)|
| 3 | H | H(B, C, D) = B ⊕ C ⊕ D |
| 4 | I | I(B, C, D) = C ⊕ (¬D ∨ B) |

#### Исходные значения переменных
|||
| --- | --- |
| A | 0x67452301 |
| B | 0xEFCDAB89 |
| C | 0x98BADCFE |
| D | 0x10325476 |

In [23]:
text = 'Исю'
encoded = text.encode('cp1251')

print('Символ | Десятичное представление | Шестнадцатеричное представление')
print('-' * 67)
for i in range(len(text)):
    print(f'{text[i]:^6} | {encoded[i]:^24} | {hex(encoded[i])[2:]:^30}')

Символ | Десятичное представление | Шестнадцатеричное представление
-------------------------------------------------------------------
  И    |           200            |               c8              
  с    |           241            |               f1              
  ю    |           254            |               fe              


In [24]:
message = 'Исю'.encode('cp1251')
messageLength = 8 * len(message)
message = bytearray(message)

message.append(0x80)
while len(message) % 64 != 56:
    message.append(0x0)
message.extend(messageLength.to_bytes(8, byteorder="little"))
print(f'hex: {message.hex()}')

hex: c8f1fe80000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001800000000000000


In [34]:
A = 0x67452301
B = 0xefcdab89
C = 0x98badcfe
D = 0x10325476

F = lambda B, C, D : (B & C) | (~B & D)
G = lambda B, C, D : (B & D) | (C & ~D)
H = lambda B, C, D : B ^ C ^ D
I = lambda B, C, D : C ^ (B | ~D)

ModuloAddition = lambda x, y : (x + y) % 0x100000000

def ki(index):
    return math.floor(abs(math.sin(index+1)) * 0x100000000)

def si(index):
    shift = [
        [7, 12, 17, 22],
        [5,  9, 14, 20],
        [4, 11, 16, 23],
        [6, 10, 15, 21]
    ]
    return shift[index // 16][index % 4]

def rol(number, offset):
    return (number << offset) | (number >> (32-offset))

computationLog = []
additionLog = []

def computation(m, compBuffer):
    q = [0] * 68
    q[0], q[1], q[2], q[3] = compBuffer[0], compBuffer[3], compBuffer[2], compBuffer[1]

    for i in range(4, 68):
        index = i - 4
        computationLog.append([hex(q[i - 4]), hex(q[i - 1]), hex(q[i - 2]), hex(q[i - 3])])
        if 0 <= index < 16:
            rf = F(q[i - 1], q[i - 2], q[i - 3])
            messageIndex = index

        elif 16 <= index < 32:
            rf = G(q[i - 1], q[i - 2], q[i - 3])
            messageIndex = (5 * index + 1) % 16
        
        elif 32 <= index < 48:
            rf = H(q[i - 1], q[i - 2], q[i - 3])
            messageIndex = (3 * index + 5) % 16
        
        else:
            rf = I(q[i - 1], q[i - 2], q[i - 3])
            messageIndex = (7 * index) % 16
        
        q[i] = ModuloAddition(q[i - 4] + rf, m[messageIndex] + ki(index))
        q[i] = rol(q[i], si(index))
        q[i] = ModuloAddition(q[i - 1], q[i]) 
        computationLog.append([hex(q[i - 3]), hex(q[i]), hex(q[i - 1]), hex(q[i - 2])])

    rA = ModuloAddition(compBuffer[0], q[64])
    rB = ModuloAddition(compBuffer[1], q[67])
    rC = ModuloAddition(compBuffer[2], q[66])
    rD = ModuloAddition(compBuffer[3], q[65])
    
    return [rA, rB, rC, rD]



def MD5(message):
    messageLength = 8 * len(message)
    message = bytearray(message)

    message.append(0x80)
    while len(message) % 64 != 56:
        message.append(0x0)
    message.extend(messageLength.to_bytes(8, byteorder="little"))

    numberOfBlocks = len(message) // 64
    compBuffer = [[0, 0, 0, 0]] * (numberOfBlocks + 1)
    compBuffer[0] = [A, B, C, D]

    for i in range(numberOfBlocks):
        curBlock = i * 64
        block = [int.from_bytes(message[curBlock + 4 * k: curBlock + 4 * (k + 1)], byteorder='little') for k in range(16)]
        compBuffer[i + 1] = computation(block, compBuffer[i])
    result = ((compBuffer[numberOfBlocks])[0].to_bytes(4, 'little') + \
              (compBuffer[numberOfBlocks])[1].to_bytes(4, 'little') + \
              (compBuffer[numberOfBlocks])[2].to_bytes(4, 'little') + \
              (compBuffer[numberOfBlocks])[3].to_bytes(4, 'little'))  \
              .hex()
    
    additionLog.append(((compBuffer[numberOfBlocks])[0].to_bytes(4) + \
                         (compBuffer[numberOfBlocks])[0].to_bytes(4) + \
                         (compBuffer[numberOfBlocks])[0].to_bytes(4) + \
                         (compBuffer[numberOfBlocks])[0].to_bytes(4))  \
                         .hex())
    additionLog.append(((compBuffer[numberOfBlocks])[0].to_bytes(4, 'little') + \
                         (compBuffer[numberOfBlocks])[1].to_bytes(4, 'little') + \
                         (compBuffer[numberOfBlocks])[2].to_bytes(4, 'little') + \
                         (compBuffer[numberOfBlocks])[3].to_bytes(4, 'little'))  \
                         .hex())

    return result

hash = MD5('Исю'.encode('cp1251'))
print(f'''Исходные значения:
A + B + C + D = {(A.to_bytes(4) + B.to_bytes(4) + C.to_bytes(4) + D.to_bytes(4)).hex()} (big-endian)
A + B + C + D = {(A.to_bytes(4, 'little') + B.to_bytes(4, 'little') + C.to_bytes(4, 'little') + D.to_bytes(4, 'little')).hex()} (little-endian)

После 4-го раунда:
A + B + C + D = {additionLog[0]} (big-endian)
A + B + C + D = {additionLog[1]} (little-endian)

Конечная хеш-сумма:
{hash}
''')

print(printTable(computationLog))


Исходные значения:
A + B + C + D = 67452301efcdab8998badcfe10325476 (big-endian)
A + B + C + D = 0123456789abcdeffedcba9876543210 (little-endian)

После 4-го раунда:
A + B + C + D = 62eb189e62eb189e62eb189e62eb189e (big-endian)
A + B + C + D = 9e18eb621e0e7b45dbd7778ab6033b99 (little-endian)

Конечная хеш-сумма:
9e18eb621e0e7b45dbd7778ab6033b99



None


In [26]:
print(hashlib.md5('Исю'.encode('cp1251')).hexdigest())

9e18eb621e0e7b45dbd7778ab6033b99


In [27]:
def printTable(computationLog):
    meh = ''
    iter = 1
    round = 1
    for i in range(len(computationLog)):
        if i % 32 == 0:
            meh += '<tr>'
            meh += '<tr></tr>'
            meh += f'<td rowspan="34">{round}</td>'
            meh += '</tr>'
            round += 1
        if i % 2 == 0:
            if iter > 16:
                iter = 1
            meh += '<tr></tr>'
            meh += f'<td>{iter}</td>'
            iter += 1
        for j in range(len(computationLog[0])):
            meh +=f'<td>{computationLog[i][j]}</td>'

    display(HTML(f'''
    <table><thead><tr><th rowspan="3">Раунд</th><th rowspan="3">Итерация</th><th colspan="8">Значения переменных</th>
    </tr><tr><th colspan="4">Начало итерации</th><th colspan="4">Конец итерации</th></tr><tr><th>A</th><th>B</th>
    <th>C</th><th>D</th><th>A</th><th>B</th><th>C</th><th>D</th></tr></thead><tbody>{meh}</tbody></table>'''))